<a href="https://colab.research.google.com/github/nutyfreshz/independent_study/blob/main/Markrov's_Chain_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

SQL:


```
--Markov's chain last 1 years as of 24/12/23

WITH cal AS
(SELECT sales.trans_date AS "date"
		, sales.member_number
		, prod.brand_name
		, sum(sales.net_price_tot) sale
FROM analysis_cds_rbs.sales_sku_cds_rbs sales
JOIN analysis_cds_rbs.ca_ds_ms_product prod
	USING(sku_id)
WHERE 1=1
	AND sales.member_number IS NOT NULL
	AND prod.dept_name IN ('BEAUTY') AND prod.class_name IN ('SKINCARE')
	AND date(sales.trans_date) BETWEEN DATEADD(YEAR,-1,GETDATE())+1 AND GETDATE()
GROUP BY 1,2,3
)

, cal_tot AS
(SELECT  t1.brand_name AS brand_from
		, t2.brand_name AS brand_to
		, count(t1.member_number) cnt
FROM cal t1, cal t2
WHERE 1=1
	AND t1.member_number = t2.member_number
	AND t1."date" < t2."date"
GROUP BY 1,2
ORDER BY 1,2
)

, fin_matrix AS
(SELECT brand_from
		, brand_to
		, cnt
		, sum(cnt) OVER(PARTITION BY brand_from) cnt_total
FROM cal_tot
GROUP BY 1,2,3
ORDER BY 1,2
)

SELECT brand_from
		, brand_to
		, cnt/cnt_total::DECIMAL(38,5) AS prob
FROM fin_matrix

;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('/content/extra_markrov.xlsx', sheet_name = 'all')
df

,brand_from,brand_to,prob
0,ACQUA ALLE ROSE,ACQUA ALLE ROSE,0.093750
1,ACQUA ALLE ROSE,AR,0.006250
2,ACQUA ALLE ROSE,BANANA BOAT,0.006250
3,ACQUA ALLE ROSE,BEAVER,0.018750
4,ACQUA ALLE ROSE,BENEFIT,0.025000
...,...,...,...
23219,ZELENS,THREE,0.017065
23220,ZELENS,VALMONT,0.001706
23221,ZELENS,WHOO,0.013652
23222,ZELENS,YSL,0.001706


# Predicting Future States

## Create Transition Matrix

In [3]:
# Get unique brands
unique_brands = sorted(set(df['brand_from']).union(set(df['brand_to'])))

# Create transition matrix with zeros
num_brands = len(unique_brands)
transition_matrix = np.zeros((num_brands, num_brands))

# Fill transition matrix based on probabilities
for index, row in df.iterrows():
    from_index = unique_brands.index(row['brand_from'])
    to_index = unique_brands.index(row['brand_to'])
    transition_matrix[from_index][to_index] = row['prob']

# Normalize transition matrix
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

print("Transition Matrix:")
print(transition_matrix)

Transition Matrix:
[[9.37500000e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.76157030e-01 0.00000000e+00 ... 0.00000000e+00
  1.37965630e-03 1.25423300e-04]
 [0.00000000e+00 2.91241940e-03 4.24381111e-02 ... 0.00000000e+00
  1.66423970e-03 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e-01
  0.00000000e+00 0.00000000e+00]
 [5.77767999e-05 2.88883799e-04 0.00000000e+00 ... 0.00000000e+00
  1.20522302e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.70648460e-03 2.55972696e-02]]


<ipython-input-3-00607aab9863>:15: RuntimeWarning: invalid value encountered in divide
  transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)


## Troubleshooting before run Matrix

'''Before run next prob: Must No NaN or Inf both 2 matrix'''

In [ ]:
# Ensure 'unique_brands' includes 'SHISEIDO'
shiseido_index = unique_brands.index('SHISEIDO')

# Extract the row for SHISEIDO from the transition matrix
shiseido_probs = transition_matrix[shiseido_index]

# Ensure the 'shiseido_probs' vector and 'transition_matrix' are correctly shaped
print("Shape of shiseido_probs:", shiseido_probs.shape)
print("Shape of transition_matrix:", transition_matrix.shape)

# Check for zeros in 'shiseido_probs' vector or 'transition_matrix'
print("Zeros in shiseido_probs:", np.count_nonzero(shiseido_probs == 0))
print("Zeros in transition_matrix:", np.count_nonzero(transition_matrix == 0))

# Check for NaN or infinite values in 'shiseido_probs' or 'transition_matrix'
print("NaN in shiseido_probs:", np.isnan(shiseido_probs).any())
print("NaN in transition_matrix:", np.isnan(transition_matrix).any())
print("Infinite values in shiseido_probs:", not np.isfinite(shiseido_probs).all())
print("Infinite values in transition_matrix:", not np.isfinite(transition_matrix).all())

# Normalize 'shiseido_probs' to handle any zero division issues
shiseido_probs_normalized = shiseido_probs / np.sum(shiseido_probs)
print("Normalized shiseido_probs:", shiseido_probs_normalized)

## Manipulate Data into Calculatable form

In [ ]:
# Addressing zero probabilities and NaN/infinite values
epsilon = 1e-8  # Small non-zero value

# Adding epsilon to transition_matrix to handle zero probabilities
transition_matrix_smoothed = transition_matrix + epsilon

# Replacing NaN or infinite values in transition_matrix with zeros or other suitable values
transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0  # Replace NaN/infinite values with 0

# Index of 'SHISEIDO' in the unique brands list
shiseido_index = unique_brands.index('SHISEIDO')

# Extract the row for SHISEIDO from the smoothed transition matrix
shiseido_probs_smoothed = transition_matrix_smoothed[shiseido_index]

# Add epsilon to shiseido_probs to handle zero probabilities
shiseido_probs_smoothed += epsilon

# Normalize 'shiseido_probs_smoothed' after handling zeros
shiseido_probs_normalized = shiseido_probs_smoothed / np.sum(shiseido_probs_smoothed)

# Predict future states (next time step) using the normalized 'shiseido_probs_normalized' and 'transition_matrix_smoothed'
next_state_vector_shiseido = np.dot(shiseido_probs_normalized, transition_matrix_smoothed)
print("Next State Probabilities for SHISEIDO (t+1):", next_state_vector_shiseido)

In [6]:
# Assuming next_state_vector_shiseido is your array
highest_prob_indices = np.argsort(-next_state_vector_shiseido)[:5]  # Get indices of top 5 highest probabilities
highest_probabilities = next_state_vector_shiseido[highest_prob_indices]

# Display top 5 highest probabilities and their corresponding indices (brands/states)
for i, index in enumerate(highest_prob_indices):
    print(f"Brand/State Index: {index}, Probability: {highest_probabilities[i]}")

Brand/State Index: 232, Probability: 0.18513758641856873
Brand/State Index: 145, Probability: 0.12645930561873053
Brand/State Index: 41, Probability: 0.07401806615758386
Brand/State Index: 64, Probability: 0.056482036347445135
Brand/State Index: 39, Probability: 0.04883585629908949


In [7]:
# Assuming next_state_vector_shiseido is your array
highest_prob_indices = np.argsort(-next_state_vector_shiseido)[:5]  # Get indices of top 5 highest probabilities
highest_probabilities = next_state_vector_shiseido[highest_prob_indices]

# Map indices to brand names
index_to_brand = {index: brand_name for index, brand_name in enumerate(unique_brands)}

# Display top 5 highest probabilities and their corresponding brands
for i, index in enumerate(highest_prob_indices):
    brand_name = index_to_brand.get(index, "Unknown Brand")
    print(f"Brand: {brand_name}, Probability: {highest_probabilities[i]}")


Brand: SHISEIDO, Probability: 0.18513758641856873
Brand: KIEHLS, Probability: 0.12645930561873053
Brand: BOBBI BROWN, Probability: 0.07401806615758386
Brand: CLARINS, Probability: 0.056482036347445135
Brand: BIOTHERM, Probability: 0.04883585629908949


# Essemble Code into Function

In [12]:
def markrov_brand(brand_name):
  # Get unique brands
  unique_brands = sorted(set(df['brand_from']).union(set(df['brand_to'])))

  # Create transition matrix with zeros
  num_brands = len(unique_brands)
  transition_matrix = np.zeros((num_brands, num_brands))

  # Fill transition matrix based on probabilities
  for index, row in df.iterrows():
    from_index = unique_brands.index(row['brand_from'])
    to_index = unique_brands.index(row['brand_to'])
    transition_matrix[from_index][to_index] = row['prob']

  # Addressing zero probabilities and NaN/infinite values

  epsilon = 1e-8  # Small non-zero value

  transition_matrix = transition_matrix / (transition_matrix.sum(axis=1, keepdims=True) + epsilon)

  ############################################################

  # Adding epsilon to transition_matrix to handle zero probabilities
  transition_matrix_smoothed = transition_matrix

  # Replacing NaN or infinite values in transition_matrix with zeros or other suitable values
  transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0  # Replace NaN/infinite values with 0

  # Index of 'SHISEIDO' in the unique brands list
  shiseido_index = unique_brands.index(brand_name)

  # Extract the row for SHISEIDO from the smoothed transition matrix
  shiseido_probs_smoothed = transition_matrix_smoothed[shiseido_index]

  # Add epsilon to shiseido_probs to handle zero probabilities
  shiseido_probs_smoothed += epsilon

  # Normalize 'shiseido_probs_smoothed' after handling zeros
  shiseido_probs_normalized = shiseido_probs_smoothed / np.sum(shiseido_probs_smoothed)

  # Predict future states (next time step) using the normalized 'shiseido_probs_normalized' and 'transition_matrix_smoothed'
  next_state_vector_shiseido = np.dot(shiseido_probs_normalized, transition_matrix_smoothed)

  ############################################################

  # Assuming next_state_vector_shiseido is your array
  highest_prob_indices = np.argsort(-next_state_vector_shiseido)[:5]  # Get indices of top 5 highest probabilities
  highest_probabilities = next_state_vector_shiseido[highest_prob_indices]

  # Map indices to brand names
  index_to_brand = {index: brand_name for index, brand_name in enumerate(unique_brands)}

  # Display top 5 highest probabilities and their corresponding brands
  for i, index in enumerate(highest_prob_indices):
    brand_name = index_to_brand.get(index, "Unknown Brand")
    print(f"Brand: {brand_name}, Probability: {highest_probabilities[i]}")

In [13]:
markrov_brand('SHISEIDO')

Brand: SHISEIDO, Probability: 0.1851380426265176
Brand: KIEHLS, Probability: 0.12645958620697365
Brand: BOBBI BROWN, Probability: 0.07401822497678473
Brand: CLARINS, Probability: 0.05648209433883934
Brand: BIOTHERM, Probability: 0.04883595808477725


In [14]:
markrov_brand('CHANEL')

Brand: CHANEL, Probability: 0.08824337302081975
Brand: DIOR, Probability: 0.08673854990925016
Brand: KIEHLS, Probability: 0.07578116746359724
Brand: CLARINS, Probability: 0.06497838480027622
Brand: LOCCITANE, Probability: 0.0638198357087771


In [15]:
markrov_brand('DIOR')

Brand: DIOR, Probability: 0.1884440531565237
Brand: CLARINS, Probability: 0.06470550781039587
Brand: KIEHLS, Probability: 0.06423866891035733
Brand: CHANEL, Probability: 0.05425867309379191
Brand: LA MER, Probability: 0.04956525255940659


In [17]:
markrov_brand('CLARINS')

Brand: CLARINS, Probability: 0.13105499174222704
Brand: KIEHLS, Probability: 0.12971549061451798
Brand: BOBBI BROWN, Probability: 0.08721423011482385
Brand: WHOO, Probability: 0.08382041526882593
Brand: ESTEE LAUDER, Probability: 0.04939058377484445


In [ ]:
import pandas as pd
import numpy as np

def markov_brand_table(df):
    result_table = pd.DataFrame(columns=['brand_name', '1st_prob_brand', '2nd_prob_brand', '3rd_prob_brand', '4th_prob_brand', '5th_prob_brand'])

    unique_brands = sorted(set(df['brand_from']).union(set(df['brand_to'])))
    epsilon = 1e-8

    num_brands = len(unique_brands)
    transition_matrix = np.zeros((num_brands, num_brands))

    for index, row in df.iterrows():
        from_index = unique_brands.index(row['brand_from'])
        to_index = unique_brands.index(row['brand_to'])
        transition_matrix[from_index][to_index] = row['prob']

    transition_matrix = transition_matrix / (transition_matrix.sum(axis=1, keepdims=True) + epsilon)

    for brand_name in unique_brands:
        transition_matrix_smoothed = transition_matrix.copy()
        transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0

        brand_index = unique_brands.index(brand_name)
        brand_probs_smoothed = transition_matrix_smoothed[brand_index]
        brand_probs_smoothed += epsilon
        brand_probs_normalized = brand_probs_smoothed / np.sum(brand_probs_smoothed)

        next_state_vector = np.dot(brand_probs_normalized, transition_matrix_smoothed)
        highest_prob_indices = np.argsort(-next_state_vector)[:5]

        index_to_brand = {index: brand for index, brand in enumerate(unique_brands)}
        top_5_brands = [index_to_brand.get(idx, "Unknown Brand") for idx in highest_prob_indices]

        result_table = result_table.append({
            'brand_name': brand_name,
            '1st_prob_brand': top_5_brands[0],
            '2nd_prob_brand': top_5_brands[1],
            '3rd_prob_brand': top_5_brands[2],
            '4th_prob_brand': top_5_brands[3],
            '5th_prob_brand': top_5_brands[4]
        }, ignore_index=True)

    return result_table

# Assuming df is your DataFrame containing brand transitions and probabilities
result_brand = markov_brand_table(df)

In [28]:
result_brand

,brand_name,1st_prob_brand,2nd_prob_brand,3rd_prob_brand,4th_prob_brand,5th_prob_brand
0,ACQUA ALLE ROSE,KIEHLS,LOCCITANE,SHISEIDO,CLARINS,DIOR
1,ACSEINE,ACSEINE,LOCCITANE,CLARINS,DIOR,KIEHLS
2,AESOP,LOCCITANE,KIEHLS,CLARINS,BOBBI BROWN,LA MER
3,ALLIE,KIEHLS,CLARINS,KANEBO,LA MER,ESTEE LAUDER
4,AMARIT,LE SKIN,JOJI SECRET YOUNG,SEWA,MOODS,SNAIL WHITE
...,...,...,...,...,...,...
286,YAIMAI,YAIMAI,NURSERY,MIIMEOW,WHOO,NUXE
287,YOD SANG,YOD SANG,CLARINS,WHOO,KIEHLS,SEWA
288,YOU,LE SKIN,SEWA,SNAIL WHITE,KIEHLS,JOJI SECRET YOUNG
289,YSL,KIEHLS,DIOR,CLARINS,ESTEE LAUDER,SHISEIDO


# Data preparation for Clustering brand behavior

In [ ]:
import pandas as pd
import numpy as np

def markov_brand_table(df):
    result_table = pd.DataFrame(columns=['brand_name', '1st_prob', '2nd_prob', '3rd_prob', '4th_prob', '5th_prob'])

    unique_brands = sorted(set(df['brand_from']).union(set(df['brand_to'])))
    epsilon = 1e-8

    num_brands = len(unique_brands)
    transition_matrix = np.zeros((num_brands, num_brands))

    for index, row in df.iterrows():
        from_index = unique_brands.index(row['brand_from'])
        to_index = unique_brands.index(row['brand_to'])
        transition_matrix[from_index][to_index] = row['prob']

    transition_matrix = transition_matrix / (transition_matrix.sum(axis=1, keepdims=True) + epsilon)

    for brand_name in unique_brands:
        transition_matrix_smoothed = transition_matrix.copy()
        transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0

        brand_index = unique_brands.index(brand_name)
        brand_probs_smoothed = transition_matrix_smoothed[brand_index]
        brand_probs_smoothed += epsilon
        brand_probs_normalized = brand_probs_smoothed / np.sum(brand_probs_smoothed)

        next_state_vector = np.dot(brand_probs_normalized, transition_matrix_smoothed)
        highest_prob_indices = np.argsort(-next_state_vector)[:5]
        highest_probabilities = next_state_vector[highest_prob_indices]

        result_table = result_table.append({
            'brand_name': brand_name,
            '1st_prob': highest_probabilities[0],
            '2nd_prob': highest_probabilities[1],
            '3rd_prob': highest_probabilities[2],
            '4th_prob': highest_probabilities[3],
            '5th_prob': highest_probabilities[4]
        }, ignore_index=True)

    return result_table

# Assuming df is your DataFrame containing brand transitions and probabilities
result_prob = markov_brand_table(df)

In [21]:
result_prob

,brand_name,1st_prob,2nd_prob,3rd_prob,4th_prob,5th_prob
0,ACQUA ALLE ROSE,0.056795,0.050906,0.050500,0.039164,0.036168
1,ACSEINE,0.457774,0.051307,0.042873,0.035006,0.034361
2,AESOP,0.119332,0.103444,0.062862,0.051998,0.048886
3,ALLIE,0.061769,0.054535,0.042875,0.037174,0.036206
4,AMARIT,0.088609,0.066614,0.066133,0.057537,0.046614
...,...,...,...,...,...,...
286,YAIMAI,0.339123,0.338285,0.242590,0.026667,0.026667
287,YOD SANG,0.059714,0.050884,0.050166,0.040792,0.037775
288,YOU,0.090828,0.052736,0.047843,0.046459,0.044481
289,YSL,0.112804,0.061837,0.061244,0.053500,0.052881


In [22]:
result_prob.to_csv('markrov_next_prob_for_clustering.csv', index = False)

# Sandbox

In [29]:
result_prob_filter = result_prob.merge(result_brand[['brand_name', '1st_prob_brand']]
                                       , how = 'inner'
                                       , on = 'brand_name'
                                       )
result_prob_filter

,brand_name,1st_prob,2nd_prob,3rd_prob,4th_prob,5th_prob,1st_prob_brand
0,ACQUA ALLE ROSE,0.056795,0.050906,0.050500,0.039164,0.036168,KIEHLS
1,ACSEINE,0.457774,0.051307,0.042873,0.035006,0.034361,ACSEINE
2,AESOP,0.119332,0.103444,0.062862,0.051998,0.048886,LOCCITANE
3,ALLIE,0.061769,0.054535,0.042875,0.037174,0.036206,KIEHLS
4,AMARIT,0.088609,0.066614,0.066133,0.057537,0.046614,LE SKIN
...,...,...,...,...,...,...,...
286,YAIMAI,0.339123,0.338285,0.242590,0.026667,0.026667,YAIMAI
287,YOD SANG,0.059714,0.050884,0.050166,0.040792,0.037775,YOD SANG
288,YOU,0.090828,0.052736,0.047843,0.046459,0.044481,LE SKIN
289,YSL,0.112804,0.061837,0.061244,0.053500,0.052881,KIEHLS


In [30]:
result_prob_filter['mstch_first'] = np.where(result_prob_filter['brand_name'] == result_prob_filter['1st_prob_brand'],
                                            'matched', 'unmatched')
result_prob_filter

,brand_name,1st_prob,2nd_prob,3rd_prob,4th_prob,5th_prob,1st_prob_brand,mstch_first
0,ACQUA ALLE ROSE,0.056795,0.050906,0.050500,0.039164,0.036168,KIEHLS,unmatched
1,ACSEINE,0.457774,0.051307,0.042873,0.035006,0.034361,ACSEINE,matched
2,AESOP,0.119332,0.103444,0.062862,0.051998,0.048886,LOCCITANE,unmatched
3,ALLIE,0.061769,0.054535,0.042875,0.037174,0.036206,KIEHLS,unmatched
4,AMARIT,0.088609,0.066614,0.066133,0.057537,0.046614,LE SKIN,unmatched
...,...,...,...,...,...,...,...,...
286,YAIMAI,0.339123,0.338285,0.242590,0.026667,0.026667,YAIMAI,matched
287,YOD SANG,0.059714,0.050884,0.050166,0.040792,0.037775,YOD SANG,matched
288,YOU,0.090828,0.052736,0.047843,0.046459,0.044481,LE SKIN,unmatched
289,YSL,0.112804,0.061837,0.061244,0.053500,0.052881,KIEHLS,unmatched


In [31]:
result_prob_filter.to_csv('markrov_next_prob_for_clustering.csv', index = False)

# Stationary Distribution

In [39]:
# Assuming transition_matrix is your transition matrix
epsilon = 1e-8
transition_matrix = transition_matrix / (transition_matrix.sum(axis=1, keepdims=True) + epsilon)

transition_matrix_smoothed = transition_matrix.copy()
transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0

# Compute eigenvalues and eigenvectors of the transpose of the transition matrix
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix_smoothed.T)

# Find the index of the eigenvalue closest to 1 (stationary distribution)
stationary_index = np.argmin(np.abs(eigenvalues - 1.0))
stationary_vector = np.abs(eigenvectors[:, stationary_index])
stationary_vector /= np.sum(stationary_vector)  # Normalize to ensure it's a probability distribution

# Display the stationary distribution
for brand_index, probability in enumerate(stationary_vector):
    brand = unique_brands[brand_index]  # Assuming unique_brands contains brand names
    print(f"Brand: {brand}, Probability: {probability}")

Brand: ACQUA ALLE ROSE, Probability: 2.227648760525039e-05
Brand: ACSEINE, Probability: 0.0011055157232044783
Brand: AESOP, Probability: 4.779462469548099e-05
Brand: ALLIE, Probability: 0.00015380741304167256
Brand: AMARIT, Probability: 0.00036015250429454246
Brand: AMPLEN, Probability: 7.598187444823172e-05
Brand: ANUA, Probability: 1.2149069683088347e-05
Brand: AR, Probability: 0.0005546538183477561
Brand: ARCHITA, Probability: 1.2454893850272234e-07
Brand: ARDERMIS, Probability: 7.427323484253901e-06
Brand: AREEYA, Probability: 1.204500710314118e-05
Brand: ARGENTUM, Probability: 2.729582288643005e-07
Brand: ARIUL, Probability: 7.040260142164544e-05
Brand: AROMATICA, Probability: 1.0705813397572963e-06
Brand: ARTY, Probability: 0.00022232773736760748
Brand: ARY PROFESSIONAL, Probability: 8.457086665341833e-06
Brand: ATREUS, Probability: 1.9732230008438265e-06
Brand: AUGUSTINUS BADER, Probability: 0.00221525874618268
Brand: AVEDA, Probability: 0.007379955484256907
Brand: AVENUE BEAUTY

In [ ]:
tab = pd.DataFrame(columns=['brand_name', 'stationary_prob'])

# Assuming transition_matrix is your transition matrix
epsilon = 1e-8
transition_matrix = transition_matrix / (transition_matrix.sum(axis=1, keepdims=True) + epsilon)

transition_matrix_smoothed = transition_matrix.copy()
transition_matrix_smoothed[~np.isfinite(transition_matrix_smoothed)] = 0

# Compute eigenvalues and eigenvectors of the transpose of the transition matrix
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix_smoothed.T)

# Find the index of the eigenvalue closest to 1 (stationary distribution)
stationary_index = np.argmin(np.abs(eigenvalues - 1.0))
stationary_vector = np.abs(eigenvectors[:, stationary_index])
stationary_vector /= np.sum(stationary_vector)  # Normalize to ensure it's a probability distribution

# Display the stationary distribution
for brand_index, probability in enumerate(stationary_vector):
    brand = unique_brands[brand_index]  # Assuming unique_brands contains brand names

    tab = tab.append({
        'brand_name': brand
        , 'stationary_prob': probability
        }, ignore_index=True)

In [52]:
tab.sort_values('stationary_prob', ascending = False)

,brand_name,stationary_prob
145,KIEHLS,0.122539
167,LOCCITANE,0.086291
281,WHOO,0.085426
41,BOBBI BROWN,0.077219
96,ESTEE LAUDER,0.049274
...,...,...
271,TOUGH & TUMBLE,0.000000
153,KRAAM,0.000000
270,TORMONTO,0.000000
185,N LIFEPLUS,0.000000


In [54]:
tab[tab['brand_name'] == 'CHANEL']

,brand_name,stationary_prob
58,CHANEL,0.02027
